Install

```
git clone https://github.com/credmark/credmark-models-py
cd credmark-models-py
pip install -e .
```

Create an .env file inside credmark-models-py folder with the following content of each chain's RPC node address:

```
CREDMARK_WEB3_PROVIDERS='{
"1":"",
"137": ",
"42161": ""
}'
```


In [1]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records, JoinType
from models.tmp_abi_lookup import UNISWAP_V3_FACTORY_ABI, UNISWAP_V3_POOL_ABI
import pandas as pd

cmf_param = {
    'chain_id': 1,  # polygon 137, arbitrum 42161, mainnet 1, bsc 56
    # set this to the latest block number of that chain.
    'block_number': 17_000_000,
    'use_local_models': '*',
}

context, _model_loader = create_cmf(cmf_param)

In [2]:
def v3_find_pool(uniswap_v3_factory, token1_addr, token2_addr, fee):
    pool_addr = uniswap_v3_factory.functions.getPool(
        token1_addr.checksum, token2_addr.checksum, fee).call()
    pool_contract = Contract(pool_addr).set_abi(
        UNISWAP_V3_POOL_ABI, set_loaded=True)
    return pool_contract


def v3_pool_events(uniswap_v3_pool, from_block, to_block, by_range=None):
    events = uniswap_v3_pool.fetch_events(
        uniswap_v3_pool.events.Swap,
        from_block=from_block, to_block=to_block, by_range=by_range)
    return pd.DataFrame(events)


In [11]:
# Fetch the Uniswap V3 pool for USDC and WETH and 0.3% fee
usdc_addr = Address('0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48')
weth_addr = Address('0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
uniswap_v3_factory = Contract('0x1F98431c8aD98523631AE4a59f267346ea31F984').set_abi(
    UNISWAP_V3_FACTORY_ABI, set_loaded=True)

usdc_weth_pool = v3_find_pool(uniswap_v3_factory, usdc_addr, weth_addr, 3000)

usdc_weth_pool, usdc_weth_pool.functions.token0().call(), usdc_weth_pool.functions.token1().call()

(Contract(address='0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'),
 '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
 '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2')

In [7]:
# reduce the range of from_block and to_block to speed up the query, some node only allow 10k blocks per query
df = v3_pool_events(usdc_weth_pool, 16_990_000, 17_000_000)

# use by_range to auto-split query into multiple queries and combine the results
df = v3_pool_events(usdc_weth_pool, 16_980_000, 17_000_000, by_range=10_000)

df.head()

,args,event,logIndex,transactionIndex,transactionHash,address,blockHash,blockNumber,sender,recipient,amount0,amount1,sqrtPriceX96,liquidity,tick
0,"(sender, recipient, amount0, amount1, sqrtPric...",Swap,217,161,b'\xa5 \x89*`p\xe4\xda+\x8b\xba\x9d\x05\xd2\x9...,0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8,b'Dh\x1a\xc3\xbc\xde\xc8\xfdi\xb2|\x03NkH\xae\...,16980152,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0xB154dce9E3BaFEC12E605feE206D55726031882F,114697623,-59858166659774681,1812660175049968765173413718181932,5243326266754707184,200769
1,"(sender, recipient, amount0, amount1, sqrtPric...",Swap,13,1,"b'\x84\xab,b\xa3\xf0\x9cp\xa9\x19\xd8\xe2dq\xc...",0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8,b'L\xe0\x83\x1d\x9f1\xfb\xf4[\xe8\xc0W\xb0mrb_...,16980167,0x6a5eF39f58C80e42642b9210Bf74f20666A2E6b6,0x6cCA86CC27EB8c7C2d10B0672FE392CFC88e62ff,-12309818,6462939878416012,1812660272413871845348643624661676,5243326266754707184,200769
2,"(sender, recipient, amount0, amount1, sqrtPric...",Swap,564,225,b'\xa8c\xc8\x17\x03\xac\x12\x88O\xde t\x0fW\xd...,0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8,b'!B\x9a\xe9\xb5\x01\x7fg\xbb-k\xdc\xe4\xf9\xf...,16980466,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,-133327360,70000000000000000,1812661323305571820626416274883445,5261563548366197669,200769
3,"(sender, recipient, amount0, amount1, sqrtPric...",Swap,166,77,b'SX\xc0;\x05\xc3\x07\x80{\x08\xe8f\x87p\x8e\x...,0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8,b'\xb3}\xf3\x14\xc3n\xaa#\xf0Z\x00\xe7\xb3\xd2...,16980506,0xEf1c6E67703c7BD7107eed8303Fbe6EC2554BF6B,0x3322f41dfa379B6D3050C1e271B0b435b3Ee3303,1013220492,-528775575451676956,1812653370777915623369717066373579,5268000192720104114,200769
4,"(sender, recipient, amount0, amount1, sqrtPric...",Swap,5,1,b'\xa4`/\x82=\xc0\xb4;\xf3\xb4\xe6p\xe1\xa1hN\...,0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8,"b""\x97\xee}n\xe7z1\xabC\x085F\x8c\xbar\x1b\x8b...",16980539,0x9507c04B10486547584C37bCBd931B2a4FeE9A41,0x9507c04B10486547584C37bCBd931B2a4FeE9A41,118755099869,-61943324322084626218,1811720862299025325788357821434541,5205308691776678705,200759


In [16]:
# Fees collected

token0 = Token(usdc_addr)
token1 = Token(weth_addr)
token0.scaled(df.query('amount0 > 0').amount0.sum() * 0.003), token1.scaled(df.query('amount1 > 0').amount1.sum() * 0.003), 

(39011.073932841005, 27.038288174247697)